In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ISP import *
from DCP import *
import rawpy

# %matplotlib notebook
%matplotlib qt5

%load_ext autoreload
%autoreload 2

In [2]:
# file_path = "./data/nikon_D850.nef"
file_path = "./data/0.ARW"


H_raw=rawpy.imread(file_path)

# 读取raw data
raw=H_raw.raw_image
print(f"RAW Shape:{raw.shape}")

OB=H_raw.black_level_per_channel[0]
white_level = H_raw.white_level
# white_level = H_raw.camera_white_level_per_channel[0]
print(f"White level {H_raw.white_level } White level per channal {H_raw.camera_white_level_per_channel} Black level {H_raw.black_level_per_channel}")

# Demosaic & BLC
img=bayer_demosaic(raw)
img[img<OB]=OB
img=(img-OB).astype('float64')/(white_level-OB)
img[img<0] = 0

# Tone mapping
# tone_curve = H_raw.tone_curve
# num = tone_curve.shape[0] # shape [65536]
# tone_curve = tone_curve.astype(float)/(num-1) 

# # Define some data points
# x = np.array([1, 2, 3, 4, 5])
# y = np.array([10, 20, 30, 40, 50])

# # Define the range for interpolation
# x_new = np.array([1.5, 2.5, 3.5, 4.5])

# # Perform linear interpolation
# y_new = np.interp(x_new, x, y)

# x = np.linspace(0,4,num)
# remapped_values = np.interp(img,x, tone_curve)

# remapped_img = remapped_values


RAW Shape:(5320, 8000)
White level 16383 White level per channal [15360, 15360, 15360, 15360] Black level [512, 512, 512, 512]


In [3]:
# img,H_raw = read_raw_simple("./data/0.ARW")

path =r'.\data\Nikon D750 Adobe Standard.xml'
# path = './data/Olympus E-M1 Mark III Adobe Standard.xml'
# path = r'E:\摄影Photo软件工具\Py-ISP\data\Nikon D850 Adobe Standard.xml'

parser = DCPParser(path)
parser.parse_matrices()
CM1 = parser.matrices['ColorMatrix1']
CM2 = parser.matrices['ColorMatrix2']
FM1 = parser.matrices['ForwardMatrix1']
FM2 = parser.matrices['ForwardMatrix2']

# Camera AWB
R_gain,G_gain,B_gain = H_raw.camera_whitebalance[0:3]
cameraWB = np.array([R_gain/G_gain,1,B_gain/G_gain])

# cameraWB = np.array(H_raw.daylight_whitebalance[0:3])

cameraNetural = 1/cameraWB # Actuall is the RGB vector represent white

print(cameraWB)

[1.83203125 1.         3.15234375]


In [8]:
# CM from exif tool
# 1243 -205 -15 -31 1209 -154 47 -47 1024
CM3 = np.array([ [1243, -205, -15],[ -31, 1209, -154],[ 47, -47, 1024]]) /1024
CM3

array([[ 1.21386719, -0.20019531, -0.01464844],
       [-0.03027344,  1.18066406, -0.15039062],
       [ 0.04589844, -0.04589844,  1.        ]])

In [10]:
# WB Temp calculation
CM = CM1
# CM = interpolateMatrix(2870,6500,3050,CM1,CM2)

XYZ2Camera = CM # AB * CM
Camer2XYZ = np.linalg.inv(CM) # inv(CM)  * inv(AB) 
# XYZ white point = Camer2XYZ * Camera netural point
# Camera netural point is (1/Camera WB) just CameraRGB response
XYZ_wp = Camer2XYZ @ cameraNetural

x,y,Y = XYZ2xyY(XYZ_wp)
xy2Temp(x,y),xyCoordToTemperature([x,y])

(np.float64(3689.0876814079106), np.float64(3709.877749933408))

In [6]:
# D50 XYZ 2 sRGB D65
#https://www.russellcottrell.com/photo/matrixCalculator.htm
XYZ2RGB = np.array(
[[ 3.1342599, -1.6171978, -0.4906852],
 [-0.9787551,  1.9161350,  0.0334462],
 [ 0.0719423, -0.2289582,  1.4052060]])

XYZ2RGB @ [0.96422,	1.00000,	0.82521] # XYZ2RGB * D50XYZ should be [1,1,1]


# XYZ to sRGB with D65 reference white
# XYZ2RGB = np.array(
# [[3.2404542, -1.5371385, -0.4985314],
# [-0.9692660,  1.8760108,  0.0415560],
# [ 0.0556434, -0.2040259,  1.0572252]])


array([0.99999995, 0.9999999 , 1.00000005])

In [11]:
# Forward Matrix
FM = FM2
# CM = interpolateMatrix(2870,6500,3050,FM1,FM2)
D = np.diag(cameraWB)
Camera2XYZD50 = FM @ D

vector, invf = im2vector(img) # [N,3]

wb_img = D @ vector.T # AWBed CameraRGB
XYZ_img = Camera2XYZD50 @ vector.T # [3,N] in D50
linear_RGB_img = XYZ2RGB @ XYZ_img # linear RGB

# Tone mapping
    # tone_curve = H_raw.tone_curve
    # num = tone_curve.shape[0] # shape [65536]
    # tone_curve = tone_curve.astype(float)/(num-1) 
    # x = np.linspace(0,1,num)
    # tonemapped_img = np.interp(linear_RGB_img,x, tone_curve)


sRGB_img = gamma(linear_RGB_img ) # sRGB


In [12]:
from PIL import Image  
rgb_raw_img = Image.fromarray(np.uint8(invf(sRGB_img.T) * 255))  # 将像素值转换为 0-255 范围内的整数
rgb_raw_img.save('./data/Nikon_FM2_RGB.jpg')  # 保存图片 

In [ ]:
plt.figure()
plt.imshow(invf(sRGB_img.T))
# plt.imshow(invf(gamma(tonemapped_img).T))

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Plotting the first image
axs[0, 0].imshow(img)
axs[0, 0].set_title('Camera RGB (No gamma)')

# Plotting the second image
axs[0, 1].imshow(invf(gamma(wb_img.T)))
axs[0, 1].set_title('AWB Camera RGB')

# Plotting the third image
axs[1, 0].imshow(invf(XYZ_img.T))
axs[1, 0].set_title('XYZ D50')

# Plotting the fourth image
axs[1, 1].imshow(invf(gamma(sRGB_img.T)))
axs[1, 1].set_title('sRGB')

plt.show()